In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

In [ ]:
# Set project ID
project_id = "rock-finder-project"
client = bigquery.Client(project=project_id)

In [ ]:
query = """
SELECT * FROM `rock-finder-project.routes.routes_gold`
"""
df = client.query(query).to_dataframe()

In [ ]:
#Pandas
import pandas as pd

In [ ]:
Cluster method 1

In [ ]:
#Rounding the latitude and longitude values to make clusters. 1 decimal us around 11 km, 2 decimals is around 1 km.
df['lat_rounded'] = df['area_latitude'].round(1)
df['lon_rounded'] = df['area_longitude'].round(1)

In [ ]:
#Group by the rouded lat & lon values
grouped = df.groupby(['lat_rounded', 'lon_rounded'])
grouped.size()

In [ ]:
Cluster method 2

In [ ]:
# Cluster using DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Assuming you have a DataFrame `df` with columns 'latitude' and 'longitude'
coords = df[['area_latitude', 'area_longitude']].to_numpy()

# Convert degrees to radians for haversine
coords_rad = np.radians(coords)

# DBSCAN with Haversine distance (great-circle distance)
kms_per_radian = 6371.0088
epsilon = 1 / kms_per_radian  # 0.5 km radius

db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(coords_rad)

# Add cluster labels back to the DataFrame
df['cluster'] = db.labels_

In [ ]:
# Check how the clusters look 
grouped = df.groupby(['cluster'])
grouped.size()

In [ ]:
# Create the new clusters table
df_clusters = df[["key","area_latitude","area_longitude","lat_rounded","lon_rounded","cluster"]]
df_clusters.head()

In [ ]:
#Prepare to upload new dataframe to Big Query
project_id = "rock-finder-project"
dataset_id = "rock-finder-project.routes"
table_id = "clusters"
full_table_id = f"{dataset_id}.{table_id}"

In [ ]:
#Upload new Dataframe to Big query 
from pandas_gbq import to_gbq
to_gbq(df_clusters, destination_table=full_table_id, project_id=project_id, if_exists="replace")